In [1]:
import pandas as pd
data  =  pd.read_csv('/Users/adham/Desktop/EMO_REC/Iemocap_clean_data.csv')
data.head()

,Unnamed: 0,session,method,gender,emotion,n_annotators,agreement,path,transcript,duration
0,0,1,script,F,neu,3,3,Session1/sentences/wav/Ses01F_script02_1/Ses01...,Fine.\n,2.070000
1,1,1,script,F,fru,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,[BREATHING]\n,1.502438
2,2,1,script,F,neu,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,That's not your flashlight.\n,2.180000
3,3,1,script,F,ang,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,You keep saying my flashlight like it's just ...,2.970000
4,4,1,script,F,ang,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,How's that supposed to make me feel?\n,1.819937


In [2]:
import os
os.chdir('/Users/adham/Desktop/EMO_REC/IEMOCAP_full_release/Sessions')

In [3]:
data['path'][0]

'Session1/sentences/wav/Ses01F_script02_1/Ses01F_script02_1_F000.wav'

In [4]:
import pandas as pd
import numpy as np
data_speech = pd.DataFrame()
data_speech['path'] = data['path']
data_speech['emotion'] = data['emotion']
data_speech['duration'] = data['duration']



In [5]:
len(data_speech)
data_speech.head()
data_speech['path'][0]

'Session1/sentences/wav/Ses01F_script02_1/Ses01F_script02_1_F000.wav'

In [ ]:
import torch
from transformers import Wav2Vec2Model, Wav2Vec2Processor
import torchaudio

# # Load the pre-trained model and processor
model = Wav2Vec2Model.from_pretrained('facebook/wav2vec2-base-960h')
processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base-960h')

# # Load your audio file as a PyTorch tensor
embeddingsList=[]
for i in range(len(data_speech)):

   audio, sample_rate = torchaudio.load(data_speech['path'][i])
   #print(audio.shape)
   # # Preprocess the audio data
   input_values = processor(audio, sampling_rate=sample_rate, return_tensors="pt").input_values
   
   # Reshape input tensor to remove extra dimension
   input_values = input_values.squeeze(0)

   # # # Generate embeddings
   with torch.no_grad():
      embeddings = model(input_values).last_hidden_state.mean(dim=1)
   #print(embeddings.shape)
   embeddingsList.append(embeddings)
   # # The embeddings variable now contains the embeddings for your audio data
   #print(embeddings.shape)
#data_speech['embeddings'] = embeddingsList

In [ ]:
os.getcwd()

'/Users/adham/Desktop/EMO_REC/IEMOCAP_full_release/Sessions'

In [ ]:
data_speech.to_csv("IemoCap_embeddings.csv")

In [ ]:
import pandas as pd
data_speech = pd.read_csv('IemoCap_embeddings.csv')

In [ ]:
data_speech.to_csv("IemoCap_embeddings.csv")

In [4]:
data_speech.head()

,Unnamed: 0,path,emotion,duration,embeddings
0,0,Session1/sentences/wav/Ses01F_script02_1/Ses01...,neu,2.070000,"tensor([[-4.2297e-02, 1.1443e-02, 1.4771e-03..."
1,1,Session1/sentences/wav/Ses01F_script02_1/Ses01...,fru,1.502438,"tensor([[-5.7967e-02, 2.9345e-02, -6.6958e-02..."
2,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,neu,2.180000,"tensor([[ 6.3062e-03, -6.7126e-03, -1.0135e-01..."
3,3,Session1/sentences/wav/Ses01F_script02_1/Ses01...,ang,2.970000,"tensor([[-3.1424e-03, -2.9699e-02, -1.4094e-01..."
4,4,Session1/sentences/wav/Ses01F_script02_1/Ses01...,ang,1.819937,"tensor([[-1.9856e-02, -2.3386e-03, -5.5584e-02..."


In [5]:
import pandas as pd
import numpy as np
import torch

# read the CSV file

# extract the string representation of the tensor from the column
embeddings_tensor=[]
for i in range(len(data_speech)):
    tensor_string = data_speech['embeddings'][i]  # assuming there is only one row in the dataframe
    #print(tensor_string[9:-3])
    # convert the string representation to a numpy array
    tensor_array = np.fromstring(tensor_string[9:-3], sep=',')

    # convert the numpy array to a PyTorch tensor
    tensor = torch.from_numpy(tensor_array)
    embeddings_tensor.append(tensor)
    # print the tensor
    #print("tensor = ",tensor)

In [13]:
len(embeddings_tensor[2])

768

In [15]:
import torch


# save the list of tensors to a file
torch.save(embeddings_tensor, 'embeddings_list.pt')

# # load the saved list of tensors
embeddings_tensor = torch.load('embeddings_list.pt')

# # print the loaded tensor list
# print(loaded_tensor_list)



In [ ]:
data_speech['embeddings'][0].shape

(6785,)

In [16]:
from sklearn.preprocessing import LabelEncoder

labels = data_speech['emotion']
# Initialize a LabelEncoder object
le = LabelEncoder()

# Fit the encoder to the labels
le.fit(labels)

# Transform the labels to categorical numbers
label_nums = le.transform(labels)

print(label_nums)

[3 2 3 ... 3 3 3]


In [5]:
label_nums

array([3, 2, 3, ..., 3, 3, 3])

In [18]:
for i in range(len(data_speech)):
   embeddings = data_speech['embeddings'][i]
   t_flat = embeddings.view(-1)
   data_speech['embeddings'][i] = t_flat
#t_flat = embeddings[0].view(-1)



AttributeError: 'str' object has no attribute 'view'

In [20]:
type(embeddings_tensor)

# convert the Series object to a PyTorch tensor
#t = torch.tensor(data_speech['embeddings'].values.tolist())

tensor_stack = torch.stack(embeddings_tensor, dim=0)

In [21]:
type(tensor_stack)

torch.Tensor